In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms
import torch.utils.data as data
from torch.utils.data.dataset import Dataset
from torchvision.models.resnet import ResNet, BasicBlock
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from skimage.color import rgb2gray
%matplotlib inline

from sklearn import metrics
import matplotlib.image as mpimg
import random
from scipy import interp
from itertools import cycle
import datetime 

import cv2
import numpy as np
from skimage import io 
from skimage.transform import rotate, AffineTransform, warp
import matplotlib.pyplot as plt
import random
from skimage import img_as_ubyte
import os
from skimage.util import random_noise

In [6]:
class ASL_Dataset(Dataset):
    def __init__(self, path_2_data, path_2_labels, train=None):
      
        # Load data from files
        data = np.load(path_2_data)
        char_labels = np.squeeze(np.load(path_2_labels))
        
        # Convert char labels to numeric
        ulabels = np.unique(char_labels)
        num_labels = np.zeros(len(char_labels))
        for i in range(len(ulabels)):
            num_labels[char_labels == ulabels[i]] = i

        ########################################################
        X = []
        for i in range(data.shape[0]):
            img = data[i]
            img_r = cv2.resize(img, dsize=(50,50), interpolation=cv2.INTER_CUBIC)
            img_g = rgb2gray(img_r)
            img_1d = img_g.flatten()
            #img_f = img_1d.reshape(1,50,50)
            X.append(img_1d)
        X = np.asarray(X)
        ######################################################## 
        if (train == True):
            data_raw = pd.read_csv("sign_mnist_train.csv")
            test_data_raw = test = pd.read_csv("sign_mnist_test.csv")

            # Break up data and labels
            labels = data_raw['label']
            data_raw.drop('label', axis=1, inplace=True)
            labels_test = test_data_raw['label']
            test_data_raw.drop('label', axis=1, inplace=True)

            # Normalize data
            data_full = data_raw.values/255
            labels_full = labels.values
            test_data_full = test_data_raw.values/255 
            labels_test_full = labels_test.values 

            # Concatenate training and test set
            X_MNIST_full = np.concatenate((data_full, test_data_full))
            y_MNIST_full = np.concatenate((labels_full, labels_test_full))

            # Get rid of unused letters and the letters c, g, and h
            X_MNIST = []
            y_MNIST = []
            for i in range(len(y_MNIST_full)):
                if (y_MNIST_full[i] <= 8 and y_MNIST_full[i] != 2 and y_MNIST_full[i] != 6 and y_MNIST_full[i] != 7): #ADDED THE LAST AND STATEMENT TO GET RID OF I's
                    X_MNIST.append(X_MNIST_full[i])
                    y_MNIST.append(y_MNIST_full[i])
            X_MNIST = np.asarray(X_MNIST)
            y_MNIST = np.asarray(y_MNIST)

          # Reshape data to 50x50
            X_MNIST_50 = []
            for i in range (len(X_MNIST)):
                img_flat_orig = X_MNIST[i];
                img_2d_orig = img_flat_orig.reshape(28, 28)
                img_2d_new = cv2.resize(img_2d_orig, dsize=(50,50), interpolation=cv2.INTER_CUBIC)
                img_1d_new = img_2d_new.flatten()
                X_MNIST_50.append(img_1d_new)
            X_MNIST_50 = np.asarray(X_MNIST_50)

            idx = np.ones((9,1457))*-1 #this will hold the index values for each letter, will be -1 if rest of row is not full
            for i in range(len(y_MNIST)):
                curr_let = y_MNIST[i]
                curr_row = idx[curr_let]
                unique_elements, counts_elements = np.unique(curr_row, return_counts=True)
                next_idx = len(unique_elements) - 1
                idx[curr_let][next_idx] = i

            X_MNIST_reduced = []
            y_MNIST_reduced = []
            for i in range(idx.shape[0]):
                if (i == 2 or i == 6 or i == 7):
                    continue
            for j in range(700): # <----------------------------- Change here to get reduced MNIST dataset size
                ran_num = random.randint(0,idx.shape[1]-1)
                while (idx[i][ran_num] == -1):
                    ran_num = random.randint(0,idx.shape[1]-1)
                X_MNIST_reduced.append(X_MNIST_50[idx[i][ran_num].astype(int)])
                y_MNIST_reduced.append(y_MNIST[idx[i][ran_num].astype(int)])
            X_MNIST_reduced = np.asarray(X_MNIST_reduced)
            y_MNIST_reduced = np.asarray(y_MNIST_reduced)

            X_comb = np.concatenate((X, X_MNIST_reduced))
            y_comb = np.concatenate((num_labels, y_MNIST_reduced))

            X_2d = []
            for i in X_comb:
                  X_2d.append(i.reshape(1, 50, 50))
            X = np.array(X_2d)
            num_labels = y_comb

        X_data = []
        for i in X:
            X_data.append(i.reshape(1,50,50))
        X = np.asarray(X_data)

    
        # Convert data & labels from numpy to PyTorch format
        data = torch.FloatTensor(X)
        #data = data.permute(0,3,1,2)        
        labels = torch.LongTensor(num_labels)        

        # Store data as part of the class        
        self.data = data
        self.labels = labels
        self.char_labels = char_labels
        
    def __len__(self):
      
        # Calculate and return the number of samples in the dataset
        return(len(self.labels))
        
    def __getitem__(self, idx):
      
        # Return the requested index of the data set as dictionary containing 
        # the data and the label
        #sample = dict()
        #sample['index'] = idx
        #sample['data'] = self.data[idx]
        #sample['label'] = self.labels[idx]
        #sample['char_label'] = self.char_labels[idx]
        
        return self.data[idx], self.labels[idx]

In [7]:
# Initialize the dataset
ASL_data = ASL_Dataset('train_data.npy', 'train_labels.npy',train=True)
val_data = ASL_Dataset("data.npy", "labels.npy",train=False)
print(ASL_data.data.shape, val_data.data.shape)
print(ASL_data.labels.shape, val_data.labels.shape)

torch.Size([2544, 1, 50, 50]) torch.Size([709, 1, 50, 50])
torch.Size([2544]) torch.Size([16993])


In [8]:
print(np.max(ASL_data.labels.numpy()))
print(np.max(val_data.labels.numpy()))
print(ASL_data.labels.numpy())

8
8
[0 0 0 ... 8 8 8]


In [10]:
# https://www.kaggle.com/justuser/sign-language-classifier-convnet-with-pytorch
class Network(nn.Module):   
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3)
        self.pool1 = nn.MaxPool2d(2)
        
        self.conv2 = nn.Conv2d(10, 15, 3) #15 or 20
        self.pool2 = nn.MaxPool2d(2)
        
        self.conv3 = nn.Conv2d(15, 20, 3) #20 or 30
        self.dropout1 = nn.Dropout2d()
        
        self.fc3 = nn.Linear(20 * 9 * 9, 270) 
        self.fc4 = nn.Linear(270, 9) 
        
        self.softmax = nn.LogSoftmax(dim=1)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout1(x)
                
        x = x.view(-1, 20 * 9 * 9) 
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        
        return self.softmax(x)
    
    
    def test(self, predictions, labels):
        
        self.eval()
        correct = 0
        for p, l in zip(predictions, labels):
            if p == l:
                correct += 1
        
        acc = correct / len(predictions)
        print("Correct predictions: %5d / %5d (%5f)" % (correct, len(predictions), acc))
        
    
    def evaluate(self, predictions, labels):
                
        correct = 0
        for p, l in zip(predictions, labels):
            if p == l:
                correct += 1
        
        acc = correct / len(predictions)
        return(acc)

In [11]:
import datetime 

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):  
        loss_train = 0.0
        for imgs, labels in train_loader: 
            outputs = model(imgs)  
            loss = loss_fn(outputs, labels.long()) 

            optimizer.zero_grad() 
            loss.backward() 
            optimizer.step()

            loss_train += loss.item() 

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch, loss_train / len(train_loader)))

In [ ]:
train_loader = DataLoader(ASL_data, batch_size=200, shuffle=True)
model = Network()
optimizer = optim.SGD(model.parameters(), lr=0.08)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 60,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2020-04-19 21:03:53.799724 Epoch 1, Training loss 2.1312367916107178
2020-04-19 21:04:16.396042 Epoch 10, Training loss 1.8167612185845008
2020-04-19 21:04:42.146597 Epoch 20, Training loss 1.4658675010387714


In [ ]:
train_loader = torch.utils.data.DataLoader(ASL_data, batch_size=200, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=1, shuffle=True)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]  
                correct += int((predicted == labels).sum()) 

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

In [ ]:
# Individual Testing
test1_img = np.load("data.npy")
test1_labels = np.load("labels.npy")
plt.imshow(test1_img[0])

In [ ]:
char_labels = np.squeeze(test1_labels)
ulabels = np.unique(char_labels)
num_labels = np.zeros(len(char_labels))
for i in range(len(ulabels)):
    num_labels[char_labels == ulabels[i]] = i
X = []
for i in range(test1_img.shape[0]):
    img = test1_img[i]
    img_r = cv2.resize(img, dsize=(50,50), interpolation=cv2.INTER_CUBIC)
    img_g = rgb2gray(img_r)
    img_1d = img_g.flatten()
    img_f = img_1d.reshape(1,50,50)
    X.append(img_f)
X = np.asarray(X)              

test_data = torch.FloatTensor(X)       
test_labels = torch.LongTensor(num_labels) 

predictions = model(test_data)
print(predictions)
torch.max(predictions.data, 1)